In [1]:
import pandas as pd
import os
import yaml
import shutil
import configparser
import cv2
import numpy as np
from tqdm import tqdm
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
def convert_to_yolo_format(bb,img_width,img_height):
    x_center=bb['bb_left']+ (bb['bb_width']/2)
    y_center=bb['bb_top'] + (bb['bb_height']/2)

    x_center/=img_width
    y_center/=img_height
    bb_width_normalized=bb['bb_width']/img_width
    bb_height_normalized=bb['bb_height']/img_height

    x_center=max(min(x_center,1),0)
    y_center = max(min(y_center, 1), 0)
    bb_width_normalized = max(min(bb_width_normalized,1),0)
    bb_height_normalized= max(min(bb_height_normalized,1),0)
    return  (x_center,y_center,bb_width_normalized,bb_height_normalized)

In [3]:
def process_folder(folder_path):
    config=configparser.ConfigParser()
    config.read(os.path.join(folder_path,'seqinfo.ini'))
    img_width=int(config['Sequence']['imWidth'])
    img_height=int(config['Sequence']['imHeight'])
    #load groud truth data
    gt_path=os.path.join(folder_path,'det/det.txt')
    gt_data=pd.read_csv(gt_path,header=None,names=['frame','id','bb_left', 'bb_top','bb_width','bb_height','conf','class','visibility'])
    labels_folder=os.path.join(folder_path,'labels')
    os.makedirs(labels_folder,exist_ok=True)

    for frame_number in gt_data['frame'].unique():
        frame_data=gt_data[gt_data['frame']==frame_number]
        label_file=os.path.join(labels_folder,f'{frame_number:06d}.txt')
        with open(label_file,'w') as file:
            for _,row in frame_data.iterrows():
                yolo_bb=convert_to_yolo_format(row,img_width,img_height)
                file.write(f'0 {yolo_bb[0]} {yolo_bb[1]} {yolo_bb[2]} {yolo_bb[3]}\n')

In [4]:

def process_all_folders(base_directory):
    for folder_name in tqdm(os.listdir(base_directory)):
        folder_path=os.path.join(base_directory,folder_name)

        if 'FRCNN' not in folder_name:
            os.system(f'rm -rf {folder_path}')
        if os.path.isdir(folder_path):
            process_folder(folder_path)

In [9]:
process_all_folders('MOT17/train')
process_all_folders('MOT17/test')

100%|██████████| 21/21 [00:16<00:00,  1.26it/s]


In [5]:
def rename_and_move_files(src_folder,dst_folder,folder_name,file_extension):
    for file_name in os.listdir(src_folder):
        if file_name.endswith(file_extension):
            new_filename=f'{folder_name}_{file_name}'
            shutil.move(os.path.join(src_folder,file_name), os.path.join(dst_folder,new_filename))

def move_files_all_folders(base_directory):
    images_dir=os.path.join(base_directory,'images')
    labels_dir=os.path.join(base_directory,'labels')
    os.makedirs(images_dir,exist_ok=True)
    os.makedirs(labels_dir,exist_ok=True)
    for folder_name in tqdm(os.listdir(base_directory)):
        if folder_name in ['images','labels']:
            continue
        folder_path=os.path.join(base_directory,folder_name)
        if os.path.isdir(folder_path):
            rename_and_move_files(os.path.join(folder_path,'img1'),images_dir,folder_name,'.jpg')
            rename_and_move_files(os.path.join(folder_path,'labels'),labels_dir,folder_name,'.txt')

def delete_subfolders ( base_directory ):
    for folder_name in os. listdir ( base_directory ):
        folder_path = os. path . join ( base_directory , folder_name )
        if os. path . isdir ( folder_path ) and folder_name not in ['images','labels']:
            shutil . rmtree ( folder_path )
            print (f" Deleted folder : { folder_name }")

In [11]:
move_files_all_folders('MOT17/train')
move_files_all_folders('MOT17/test')

100%|██████████| 9/9 [00:10<00:00,  1.15s/it]


In [12]:
delete_subfolders ('MOT17/train')
delete_subfolders ('MOT17/test')

 Deleted folder : MOT17-02-FRCNN
 Deleted folder : MOT17-04-FRCNN
 Deleted folder : MOT17-05-FRCNN
 Deleted folder : MOT17-09-FRCNN
 Deleted folder : MOT17-10-FRCNN
 Deleted folder : MOT17-11-FRCNN
 Deleted folder : MOT17-13-FRCNN
 Deleted folder : MOT17-01-FRCNN
 Deleted folder : MOT17-03-FRCNN
 Deleted folder : MOT17-06-FRCNN
 Deleted folder : MOT17-07-FRCNN
 Deleted folder : MOT17-08-FRCNN
 Deleted folder : MOT17-12-FRCNN
 Deleted folder : MOT17-14-FRCNN


In [19]:
class_label=['object']
data_root_dir=os.path.join(os.getcwd(),'MOT17')
yolo_yaml_path=os.path.join(data_root_dir,'mot17_data.yml')
data_yaml={
    'path':data_root_dir,
    'train':'train/images',
    'val':'test/images',
    'nc': len(class_label),
    'names':class_label
}
with open(yolo_yaml_path,'w') as f:
    yaml.dump(data_yaml,f,default_flow_style=False)

In [7]:
import ultralytics
#ultralytics.checks()
from ultralytics import YOLO

e:\AIO2023\object_tracking_project\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
import torch
print(torch.cuda.is_available())

True


In [5]:
model=YOLO('yolov8s.pt')

epochs=30
batch_size=-1
img_size=640
project_name='models/yolo'
name='yolov8s_mot17_det'


results=model.train(
    data=yolo_yaml_path,
    epochs=epochs,
    batch=batch_size,
    imgsz=img_size,
    project=project_name,
    name=name
)

Ultralytics YOLOv8.1.27 🚀 Python-3.11.8 torch-2.2.1 CUDA:0 (NVIDIA GeForce GTX 1070, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=e:\AIO2023\object_tracking_project\MOT17\mot17_data.yml, epochs=30, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=models/yolo, name=yolov8s_mot17_det4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, 

train: Scanning E:\AIO2023\object_tracking_project\MOT17\train\labels... 5316 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5316/5316 [01:19<00:00, 66.81it/s] 


train: New cache created: E:\AIO2023\object_tracking_project\MOT17\train\labels.cache


val: Scanning E:\AIO2023\object_tracking_project\MOT17\test\labels... 5908 images, 11 backgrounds, 0 corrupt: 100%|██████████| 5919/5919 [01:53<00:00, 52.05it/s] 


val: New cache created: E:\AIO2023\object_tracking_project\MOT17\test\labels.cache
Plotting labels to models\yolo\yolov8s_mot17_det4\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.00044531249999999996), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to models\yolo\yolov8s_mot17_det4
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      30/30      5.16G     0.5163      0.289     0.8144        226        640: 100%|██████████| 280/280 [03:02<00:00,  1.53it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 156/156 [00:57<00:00,  2.72it/s]

                   all       5919     110141      0.896      0.823      0.906      0.649



30 epochs completed in 2.206 hours.
Optimizer stripped from models\yolo\yolov8s_mot17_det4\weights\last.pt, 22.5MB
Optimizer stripped from models\yolo\yolov8s_mot17_det4\weights\best.pt, 22.5MB

Validating models\yolo\yolov8s_mot17_det4\weights\best.pt...
Ultralytics YOLOv8.1.27 🚀 Python-3.11.8 torch-2.2.1 CUDA:0 (NVIDIA GeForce GTX 1070, 8192MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 156/156 [01:25<00:00,  1.83it/s]


                   all       5919     110141      0.896      0.823      0.906      0.649
Speed: 0.1ms preprocess, 5.1ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to models\yolo\yolov8s_mot17_det4


In [8]:
class YOLOv8 :
    def __init__ (self ,model_path):
        self . model = YOLO ( model_path )
    def detect (self , source_img ):
        results = self.model.predict(source_img,verbose = False )[0]
        bboxes = results.boxes.xywh.cpu().numpy()
        bboxes [:, :2] = bboxes [:, :2] - ( bboxes [:, 2:] / 2)
        scores = results.boxes.conf.cpu().numpy()
        class_ids = results.boxes.cls.cpu().numpy()
        return bboxes , scores , class_ids

In [2]:
!git clone https://github.com/wjnwjn59/deep_sort.git

Cloning into 'deep_sort'...


In [9]:
from deep_sort.deep_sort import nn_matching
from deep_sort.deep_sort.detection import Detection
from deep_sort.deep_sort.tracker import Tracker
from deep_sort.tools import generate_detections as gdet

In [10]:
class DeepSORT :
    def __init__ (self ,model_path ='resources/networks/mars-small128.pb',
                  max_cosine_distance = 0.7,
                  nn_budget = None ,
                  classes =['objects']
                  ):
        self . encoder = gdet . create_box_encoder ( model_path , batch_size =1)
        self . metric = nn_matching . NearestNeighborDistanceMetric ('cosine',max_cosine_distance , nn_budget )
        self . tracker = Tracker ( self . metric )
        key_list = []
        val_list = []
        for ID , class_name in enumerate ( classes ):
            key_list . append (ID)
            val_list . append ( class_name )
        self . key_list = key_list
        self . val_list = val_list
    
    def tracking (self , origin_frame , bboxes , scores , class_ids ):
        features = self . encoder ( origin_frame , bboxes )
        detections = [ Detection (bbox , score , class_id , feature )
        for bbox , score , class_id , feature in zip(bboxes ,scores ,class_ids ,features )]
        self . tracker . predict ()
        self . tracker . update ( detections )

        tracked_bboxes = []
        for track in self . tracker . tracks :
            if not track . is_confirmed () or track . time_since_update > 5:
                continue
            bbox = track . to_tlbr ()
            class_id = track . get_class ()
            conf_score = track . get_conf_score ()
            tracking_id = track . track_id
            tracked_bboxes . append (bbox . tolist () + [ class_id , conf_score , tracking_id ])

        tracked_bboxes = np. array ( tracked_bboxes )
        return tracked_bboxes

In [11]:
def draw_detection(img,bboxes,scores,class_ids,ids,classes=['object'],mask_alpha=0.3):
    height,width=img.shape[:2]
    np.random.seed(0)
    rng=np.random.default_rng(3)
    colors=rng.uniform(0,255,size=(len(classes),3))
    mask_img=img.copy()
    det_img=img.copy()
    size=min([height,width])*0.0006
    text_thickness=int(min([height,width])*0.001)

    for bbox,score,class_id,id_ in zip(bboxes,scores,class_ids,ids):
        color=colors[class_id]
        x1,y1,x2,y2=bbox.astype(int)

        cv2.rectangle(det_img,(x1,y1),(x2,y2),color,2)
        cv2.rectangle(mask_img,(x1,y1),(x2,y2),color,-1)
        label=classes[class_id]
        caption=f'{label} {int(score*100)}% ID: {id_}'
        (tw,th),_=cv2.getTextSize(text=caption,fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=size,thickness=text_thickness)
        th=int(th*1.2)
        cv2 . rectangle ( det_img , (x1 , y1), (x1 + tw , y1 - th), color , -1)
        cv2 . rectangle ( mask_img , (x1 , y1), (x1 + tw , y1 - th), color , -1)
        cv2 . putText ( det_img , caption , (x1 , y1),
                       cv2 . FONT_HERSHEY_SIMPLEX , size ,
                       (255 , 255 , 255) ,
                       text_thickness , cv2 . LINE_AA )
        cv2 . putText ( mask_img , caption , (x1 , y1),
                       cv2 . FONT_HERSHEY_SIMPLEX , size ,(255 , 255 , 255) ,
                       text_thickness , cv2 . LINE_AA )
    return cv2. addWeighted ( mask_img , mask_alpha , det_img , 1 - mask_alpha , 0)

In [12]:
def video_tracking ( video_path , detector , tracker ,is_save_result =False , save_dir ='tracking_results'):
    cap = cv2. VideoCapture ( video_path )
    width = int(cap.get(cv2. CAP_PROP_FRAME_WIDTH ))
    height = int (cap. get(cv2. CAP_PROP_FRAME_HEIGHT ))
    if is_save_result :
        os. makedirs ( save_dir , exist_ok = True )
        # Get the video properties
        fps = int(cap.get (cv2. CAP_PROP_FPS ))

        # Define the codec and create the video writer
        fourcc = cv2 . VideoWriter_fourcc (* 'MJPG')
        save_result_name = 'output_video.avi'
        save_result_path = os. path . join ( save_dir , save_result_name )
        out = cv2. VideoWriter ( save_result_path , fourcc , fps , (width , height ))
    
    all_tracking_results = []
    tracked_ids = np. array ([] , dtype =np. int32 )
    while True :
        ret , frame = cap. read ()
        if not ret:
            break
        detector_results = detector . detect ( frame )
        bboxes , scores , class_ids = detector_results
        tracker_pred = tracker .tracking (origin_frame =frame ,bboxes =bboxes ,scores =scores ,class_ids = class_ids)

        if tracker_pred . size > 0:
            bboxes = tracker_pred [:, :4]
            class_ids = tracker_pred [:, 4]. astype (int )
            conf_scores = tracker_pred [:, 5]
            tracking_ids = tracker_pred [:, 6]. astype (int)

        # Get new tracking IDs
            new_ids = np. setdiff1d ( tracking_ids , tracked_ids )

        # Store new tracking IDs
            tracked_ids = np. concatenate (( tracked_ids , new_ids ))
            result_img = draw_detection (img =frame ,bboxes =bboxes ,scores = conf_scores ,class_ids = class_ids ,ids = tracking_ids)
        else :
            result_img = frame
        all_tracking_results . append ( tracker_pred )
        if is_save_result == 1:
            out . write ( result_img )
        # Break the loop if ’q’ is pressed
        if cv2. waitKey (25) & 0xFF == ord('q'):
            break

        # Release video capture
    cap . release ()
    if is_save_result :
        out . release ()
        cv2 . destroyAllWindows ()
    return all_tracking_results

In [20]:
yolo_model_path = 'models\yolo\yolov8s_mot17_det4\weights\\best.pt'

detector = YOLOv8 ( yolo_model_path )
tracker = DeepSORT ()
video_path = 'video.mp4'
all_tracking_results = video_tracking (video_path ,detector ,tracker ,is_save_result = True)

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os

# Input video path
output_video_path = 'tracking_results/output_video.avi'

# Compressed video path
compressed_path = 'tracking_results/result_compressed.mp4'

os.system(f"ffmpeg -i {output_video_path} -vcodec libx264 {compressed_path}")
